# Padrão Strategy usando funções

**Definição**. Padrão que:
1. define uma família de algoritmos,
2. encapsuca dada um,
3. torna-os intercambiáveis.

## Ator e Objeto

In [39]:
from collections import namedtuple


Cliente = namedtuple('Cliente', 'nome fidelidade')

class ProdutoNoCarrinho:
    def __init__(self, nome, quantidade, preco) -> None:
        self.nome = nome
        self.quantidade = quantidade
        self.preco = preco

    def total(self):
        return self.preco * self.quantidade

## Contexto

In [40]:
class Pedido:
    def __init__(self, cliente: Cliente, carrinho: list[ProdutoNoCarrinho], promocao=None) -> None:
        self.cliente = cliente
        self.carrinho = carrinho
        self.promocao = promocao
        self.desconto = 0.0

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.carrinho)
        return self.__total

    def desconte(self):
        if self.promocao is not None:
            self.desconto = self.promocao(self)

    def total_considerando_descontos(self):
        if self.promocao is None:
            return self.total()
        self.desconte()
        return self.total() - self.desconto

    def __repr__(self) -> str:
        self.desconte()
        linhas = [
            ("Subtotal",  f"  R$ {self.total():7.2f}"),
            (" Desconto", f"- R$ {self.desconto:7.2f}"),
            ("    Total", f"  R$ {self.total_considerando_descontos():7.2f}"),
        ]
        return "<" + '\n'.join(f"{tipo}: {valor}" for tipo,valor in linhas) + ">"

## Interface da estratégia

In [41]:
# import abc

# class Promocao(abc.ABC):
#     @abc.abstractmethod
#     def desconto(self, pedido: Pedido) -> float: pass

## Estratégias (como funções)

In [42]:
def fidelidade_promo(pedido: Pedido) -> float:
    if pedido.cliente.fidelidade < 1000:
        return 0
    return pedido.total() * 0.05

def combinados_promo(pedido: Pedido) -> float:
    itens_elegiveis = [item for item in pedido.carrinho if item.quantidade >= 20]
    return sum(item.total() for item in itens_elegiveis) / 10

def pedido_grande_promo(pedido: Pedido) -> float:
    if len({item.nome for item in pedido.carrinho}) >= 10:
        return pedido.total() * 0.07
    return 0

## Testando...

In [43]:
joe = Cliente("Joe", 0)
ann = Cliente("Anna", 1100)

carrinho = [
    ProdutoNoCarrinho("Banana", 4, .5),
    ProdutoNoCarrinho("Maçã", 10, 1.5),
    ProdutoNoCarrinho("Melancia", 5, 5),
]

In [44]:
Pedido(joe, carrinho, fidelidade_promo)

<Subtotal:   R$   42.00
 Desconto: - R$    0.00
    Total:   R$   42.00>

In [45]:
Pedido(ann, carrinho, fidelidade_promo)

<Subtotal:   R$   42.00
 Desconto: - R$    2.10
    Total:   R$   39.90>

## Encontrando a melhor estratégia

In [46]:
Pedido(joe, carrinho, fidelidade_promo)

<Subtotal:   R$   42.00
 Desconto: - R$    0.00
    Total:   R$   42.00>

In [47]:
Pedido(joe, carrinho, combinados_promo)

<Subtotal:   R$   42.00
 Desconto: - R$    0.00
    Total:   R$   42.00>

In [48]:
Pedido(joe, carrinho, pedido_grande_promo)

<Subtotal:   R$   42.00
 Desconto: - R$    0.00
    Total:   R$   42.00>

In [52]:
def melhor_promocao(pedido: Pedido) -> float:
    promos = [fidelidade_promo, combinados_promo, pedido_grande_promo]
    return max(promo(pedido) for promo in promos)

In [53]:
Pedido(ann, carrinho, melhor_promocao)

<Subtotal:   R$   42.00
 Desconto: - R$    2.10
    Total:   R$   39.90>